In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
from termcolor import colored
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from natsort import natsorted

In [3]:
# Display pandas df without truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Simulated Results!

In [4]:
mainFolderPath = '/home/luigi/Documents/ARGoS/argos-LMCRW-script/results/ALL/Simulated/baseline'

In [5]:
for dirName, subdirList, fileList in os.walk(mainFolderPath):
    if not subdirList:
        print(dirName)
#         print('\t', dirName.split("/")[-2])
#         print('\t', dirName.split("/")[-1])
        
        exp_type = dirName.split("/")[-2]
        params = dirName.split("/")[-1]
        fullPosition_array = np.array([])
        
        
        for filename in os.listdir(dirName):
            if filename.endswith('position.tsv'):
                if not os.path.getsize(os.path.join(dirName, filename)) > 0:
                    print(colored("Error, empty file at:" + os.path.join(dirName, filename), 'red'))
                    continue
#                 print('\t', filename)
                df_single = pd.read_csv(os.path.join(dirName, filename), sep="\t")
                [num_times, num_robots] = df_single.T.values[1:].shape
#                 print('num_times {}, num_robots {}'.format(num_times, num_robots), end='\n\n')
                pos_2d = np.array([x.split(',') for x in df_single.T.values[1:].ravel()], dtype=float)
                pos_2d = pos_2d.reshape(num_times, num_robots, -1)

                fullPosition_array = np.column_stack([fullPosition_array, pos_2d]) if fullPosition_array.size else pos_2d
                
#         print(fullPosition_array.shape)
        fileName = exp_type+'_'+params+'simulated_pos.npy'
        np.save(os.path.abspath(os.path.join(mainFolderPath, os.pardir))+'/'+fileName, fullPosition_array)
        print("SAVING, ", fileName)
    

/home/luigi/Documents/ARGoS/argos-LMCRW-script/results/ALL/Simulated/baseline/2022-03-18_robots#1_alpha#1.8_rho#0.0_baseline_1800
SAVING,  baseline_2022-03-18_robots#1_alpha#1.8_rho#0.0_baseline_1800simulated_pos.npy
/home/luigi/Documents/ARGoS/argos-LMCRW-script/results/ALL/Simulated/baseline/2021-10-13_robots#1_alpha#1.2_rho#0.3_baseline_1800
SAVING,  baseline_2021-10-13_robots#1_alpha#1.2_rho#0.3_baseline_1800simulated_pos.npy
/home/luigi/Documents/ARGoS/argos-LMCRW-script/results/ALL/Simulated/baseline/2022-03-18_robots#1_alpha#1.8_rho#0.9_baseline_1800
Error, empty file at:/home/luigi/Documents/ARGoS/argos-LMCRW-script/results/ALL/Simulated/baseline/2022-03-18_robots#1_alpha#1.8_rho#0.9_baseline_1800/seed#404_position.tsv
Error, empty file at:/home/luigi/Documents/ARGoS/argos-LMCRW-script/results/ALL/Simulated/baseline/2022-03-18_robots#1_alpha#1.8_rho#0.9_baseline_1800/seed#265_position.tsv
SAVING,  baseline_2022-03-18_robots#1_alpha#1.8_rho#0.9_baseline_1800simulated_pos.npy
/ho

In [ ]:
# fullPosition_array = np.load('results/'+exp_type+'_'+params+'_simulated_pos.npy')

# Real Results!

In [ ]:
def check_float(potential_float):
    try:
        float(potential_float)
        return True

    except ValueError:
        return False

### ARK —> ARGoS coordinate system

In [ ]:
ARENA_SIZE = 1875
M_TO_PIXEL = 20
CM_TO_M = 100

In [ ]:
def x_transform(x_array):
    x_trans = ((x_array - (ARENA_SIZE/2))/ M_TO_PIXEL)/CM_TO_M
    return x_trans
    

In [ ]:
def y_transform(y_array):
    y_trans = (((ARENA_SIZE/2) - y_array)/ M_TO_PIXEL) /CM_TO_M
    return y_trans

### Exp parameters

In [ ]:
fieldsPerKilobot = 6
experiment_length = 1800

In [ ]:
def generate_kiloLOG_label(n_rob):
    kiloLOG_label = ['time']
    for i in range(0, n_rob*fieldsPerKilobot, fieldsPerKilobot):
        kiloLOG_label += ['id' + str(i // fieldsPerKilobot), 'col' + str(i // fieldsPerKilobot), 'posx' + str(i // fieldsPerKilobot), 'posy' + str(i // fieldsPerKilobot), 'ori' + str(i // fieldsPerKilobot), 'state' + str(i // fieldsPerKilobot)]
    
    return kiloLOG_label

In [ ]:
def generate_kiloPOS_label(n_rob):
    kiloPOS_label = []
    for i in range(n_rob):
        kiloPOS_label += ["posx"+str(i),"posy"+str(i)]
        
    return kiloPOS_label

In [ ]:
folder_test = ['results/JAN_2022', 
               'results/FEB_2022_10#robots_TEST', 
               'results/FEB_2022_5#robots_TEST', 
               'results/FEB_2022_5#robots_TEST1', 
               'results/FEB_2022_5#robots_rho#0.0_TEST', 
               'results/FEB_2022_5#robots_rho#0.9_oldTable',
               'results/FEB_2022_5#robots_rho#0.9_oldTable_noRing',
               'results/FEB_2022_5#robots_rho#0.9_newTable_exactLinVel',
               'results/FEB_2022_5#robots_rho#0.9_newTable_BUG',
               'results/FEB_2022_5#robots_rho#0.9_oldTable_BUG',
               'results/FEB_2022_10#robots_rho#0.9_oldTable_BUGMIXED',
               'results/FEB_2022_5#robots_rho#0.9_oldTable_BUG_wrongCalib',
               'results/FEB_2022_5#robots_rho#0.9_oldTable_BUG_correctCalib',
               'results/FEB_2022_5#robots_rho#0.9_newBatteries_TEST',
               'results/FEB_2022_20#robots_rho#0.9_oldBatteries_TEMP']

In [ ]:
folder_5Minutes = ['results/FEB_2022_5MinutesTest']

In [ ]:
main_folder = os.getcwd()
folder_experiments = 'results/ALL/Real'

num_point_in_time = 181

if(folder_experiments in folder_test):
    num_point_in_time = 61
    
elif(folder_experiments in folder_5Minutes):
    num_point_in_time = 31
    
    
print("Exp lenght: ", num_point_in_time)

In [ ]:
for dirName, subdirList, fileList in natsorted(os.walk(os.path.join(main_folder,folder_experiments))):
    
    num_robots = "-1"
    rho = -1.0
    alpha = -1.0
    elements = dirName.split("/")[-1].split("_")
    for e in elements:
        if e.endswith("robots"):
            num_robots = e.split("#")[0]

        if (e.startswith("rho")):
            if check_float(e.split("#")[-1]): rho = float(e.split("#")[-1])
        if (e.startswith("alpha")):
            if check_float(e.split("#")[-1]): alpha = float(e.split("#")[-1])
    
    if num_robots == "-1" or rho == -1.0 or alpha == -1:
        continue
    
    print(colored("DirName:", 'blue'),dirName)
    print('num_robots {}, rho {}, alpha {}'.format(num_robots, rho, alpha), end='\n\n')
        
        
    fullPosition_array = np.array([])
    
    for filename in natsorted(fileList):
        if filename.endswith('kilopos.txt'):
            print('\t', filename)
            if not os.path.getsize(os.path.join(dirName, filename)) > 0:
                print("Error, empty file at:" + os.path.join(fileList, filename))
                continue
            df_single = pd.read_csv(os.path.join(dirName, filename), sep="\t", header=None)
            kiloLOG_label = generate_kiloLOG_label(int(num_robots))
            df_single.columns = kiloLOG_label
#             df_single = df_single[df_single['time'] < experiment_length+1]
            kiloPOS_label = generate_kiloPOS_label(int(num_robots))
        
            #take each tenth-row, and select [0,1800] time interval
            df_selection = df_single[kiloPOS_label].iloc[::10].iloc[:num_point_in_time]
            #così è diviso in ordine del tempo
            np_position = df_selection.to_numpy()
            np_position[:,::2] = x_transform(np_position[:,::2])
            np_position[:,1::2] = y_transform(np_position[:,1::2])
            np_position = np_position.reshape(df_selection.shape[0], int(num_robots), -1)
            fullPosition_array = np.column_stack([fullPosition_array, np_position]) if fullPosition_array.size else np_position
        
    print(fullPosition_array.shape)
    fileName = 'robots#'+str(num_robots)+'_alpha#'+str(alpha)+'_rho#'+str(rho)+'_real_pos.npy'
    filePath = os.path.join(main_folder,folder_experiments, fileName)
    print('Saving: ', filePath)
    np.save(filePath, fullPosition_array)
        